In [30]:
import librosa as lb
import numpy as np

In [31]:
from static.src import PREDICT, vggish

In [26]:
y,fr = lb.load('static/files/recordedAudio.wav')

In [35]:
xdemo = PREDICT.predict(y,fr)

TypeError: ('Keyword argument not understood:', 'input')

In [ ]:
xdemo

In [ ]:
import keras

In [3]:
import tensorflow as tf

In [4]:
import keras

In [5]:
from keras.models import Model

In [7]:
from keras import engine

In [12]:
from keras.utils.layer_utils import get_source_inputs

In [20]:
sound_model = vggish.VGGISH()

In [21]:
sound_model.input

<KerasTensor: shape=(None, 256, 64, 1) dtype=float32 (created by layer 'input_1')>

In [22]:
x = sound_model.get_layer(name="full_connect2").output 

In [23]:
ext = Model(sound_model.input,x)

In [32]:
NUM_FRAMES = 256  # Frames in input mel-spectrogram patch.
NUM_BANDS = 64  # Frequency bands in input mel-spectrogram patch.
EMBEDDING_SIZE = 128  # Size of embedding layer.

# Hyperparameters used in feature and example generation.
SAMPLE_RATE = 16000
STFT_WINDOW_LENGTH_SECONDS = 0.025
STFT_HOP_LENGTH_SECONDS = 0.010
NUM_MEL_BINS = NUM_BANDS
MEL_MIN_HZ = 125
MEL_MAX_HZ = 7500
LOG_OFFSET = 0.01  # Offset used for stabilized log of input mel-spectrogram.
EXAMPLE_WINDOW_SECONDS = 2.56  # Each example contains 96 10ms frames
EXAMPLE_HOP_SECONDS = 2.56 

n_fft = 2 ** int(np.ceil(np.log(int(round(SAMPLE_RATE*STFT_WINDOW_LENGTH_SECONDS)))/np.log(2.0)))


features_sample_rate = 1.0 / STFT_HOP_LENGTH_SECONDS
example_window_length = int(round(
    EXAMPLE_WINDOW_SECONDS * features_sample_rate))
example_hop_length = int(round(
    EXAMPLE_HOP_SECONDS * features_sample_rate))

def frame(data,window_length,hop_length):
    num_samples = data.shape[0]
    num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
    shape = (1, window_length) + data.shape[1:]
    strides = (data.strides[0] * hop_length,) + data.strides
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

In [33]:
mel_spectrogram = lb.feature.melspectrogram(y,sr=SAMPLE_RATE,
        n_fft= int(n_fft),
        hop_length= round(STFT_HOP_LENGTH_SECONDS*SAMPLE_RATE),
        win_length=round(STFT_WINDOW_LENGTH_SECONDS*SAMPLE_RATE),
        n_mels=NUM_MEL_BINS,
        fmax=MEL_MAX_HZ,
        fmin=MEL_MIN_HZ,
        window='hann'
        )
mel_spectrogram = mel_spectrogram.T

log_mel_spectrogram = np.log(mel_spectrogram)

log_mel_spectrogram = frame(log_mel_spectrogram,window_length=example_window_length,
    hop_length=example_hop_length)
    
spectrogram = np.expand_dims(log_mel_spectrogram,3)

features = ext.predict(spectrogram)

In [34]:
features.shape

(1, 128)